<a href="https://colab.research.google.com/github/gomesluiz/pln-na-pratica/blob/main/01-aquisicao-dados-enriquecimento-de-dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aquisição de dados com raspagem de dados**

In [ ]:
# Instalação dos pacotes necessários.
!pip install beautifulsoup4 pandas requests

In [ ]:
# Importação de pacotes básicos para a execução deste notebook.
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup

print("Pacotes importados com sucesso.")

In [ ]:
# Define o diretório local para armazenar as bases de dados públicas coletadas.
corpora_caminho = "./corpora"

# Verifica se o diretório especificado já existe no ambiente do Colab.
if not os.path.exists(corpora_caminho):
  # Caso o diretório não exista, cria um novo diretório com o nome 'corpora'.
  # Isso é útil para organizar os arquivos de dados baixados ou gerados.
  os.mkdir(corpora_caminho)

print(f"Diretório {corpora_caminho} criado com sucesso.")


## 1. G1

## Raspagem de dados de notícias do G1

O g1 é um portal de notícias brasileiro mantido pelo Grupo Globo e sob orientação da Central Globo de Jornalismo. Foi lançado em 18 de setembro de 2006, ano que a TV Globo fez 41 anos. O portal disponibiliza o conteúdo de jornalismo das diversas empresas do Grupo Globo - TV Globo, GloboNews, rádio CBN, Jornais O Globo, Extra e Valor Econômico, revista e Globo Rural, entre outras - além de reportagens próprias em formato de texto, fotos, áudio e vídeo. Além das cinco redações próprias situadas no Rio de Janeiro, em São Paulo, Brasília, Belo Horizonte e Recife, afiliadas da TV Globo, jornais, revistas, rádios e as agências de Notícias como Agência Estado, Agência France Presse, Associated Press, EFE, New York Times, Lusa, Reuters e Valor Econômico alimentam o portal de notícias, que é atualizado 24 horas por dia.

In [ ]:
# Define a URL da base pública armazenada no Github.
url = "https://g1.globo.com/"

In [ ]:
# Configuração do arquivo para armazenamento da base de dados.
noticias_arquivo = "g1_noticias.csv"

# O método os.path.join cria o caminho completo para o arquivo, combinando
# 'corpora_caminho' com o nome do arquivo 'b2w_arquivo'.
noticias_caminho = os.path.join(corpora_caminho, noticias_arquivo)

# Executa o download do arquivo especificado na URL com os parâmetros
# especificados
noticias_g1 = []

resposta = requests.get(url)
if resposta.status_code == 200:
  # Se o status for igual a 200 (sucesso), grava o arquivo no caminho
  # especificado
  pagina = resposta.content
  soup = BeautifulSoup(pagina, 'html.parser')
  noticias = soup.findAll('div', attrs={'class': 'feed-post-body'})

  for noticia in noticias:
    # Título
    titulo = noticia.find('a', attrs={'class': 'feed-post-link'})

    # Subtítulo: div class="feed-post-body-resumo"
    subtitulo = noticia.find('div', attrs={'class': 'feed-post-body-resumo'})

    if (subtitulo):
      noticias_g1.append([titulo.text, subtitulo.text, titulo['href']])
    else:
      noticias_g1.append([titulo.text, '', titulo['href']])

  noticias_g1_df = pd.DataFrame(noticias_g1, columns=['Título', 'Subtítulo', 'Link'])

  noticias_g1_df.to_csv(noticias_caminho, index=False)

  print(f"Raspagem de dados da URL {url} gravada em {noticias_arquivo} com sucesso!")
else:
    print("Falha na solicitação, código de status:", resposta.status_code)



In [ ]:
# O comando head do linux mostra as cinco primeiras linhas do arquivo gravado.
noticias_g1_df = pd.read_csv(noticias_caminho)
noticias_g1_df.head(10)